In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

In [5]:
tickers = ["SPY", "GME", "AMC", "TSLA", "BB", "NVDA", "BBBY", "MVIS", "BABA", "AMD"]
ohlc = yf.download(tickers, period="max")
prices = ohlc["Adj Close"]
prices.tail()

[*********************100%***********************]  10 of 10 completed


,AMC,AMD,BABA,BB,BBBY,GME,MVIS,NVDA,SPY,TSLA
Date,,,,,,,,,,
2021-05-17,13.95,74.650002,211.050003,8.49,25.200001,180.600006,15.19,566.619995,415.519989,576.830017
2021-05-18,14.03,74.440002,213.720001,8.77,24.840000,180.669998,15.08,560.630005,411.940002,577.869995
2021-05-19,12.64,76.230003,212.539993,8.54,24.650000,168.830002,13.98,562.630005,410.859985,563.460022
2021-05-20,12.55,78.059998,216.990005,8.78,23.690001,170.490005,14.34,584.500000,415.279999,586.780029
2021-05-21,12.08,77.169998,211.059998,8.52,24.150000,176.789993,13.87,599.669983,414.940002,580.880005


In [6]:
prices.head()

,AMC,AMD,BABA,BB,BBBY,GME,MVIS,NVDA,SPY,TSLA
Date,,,,,,,,,,
1980-03-17,NaN,3.145833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-18,NaN,3.031250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-19,NaN,3.041667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-20,NaN,3.010417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-21,NaN,2.916667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
prices = prices[prices.index >= "2017-08-01"]

In [ ]:
prices.to_csv("stock_prices.csv")